In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle

## Game Logic

In [ ]:
board = np.full((3, 3), 0 ,dtype=int)
player = 1

def make_move(board, move):
    global player
    x,y = move[0], move[1]
    if board[y][x] == 0:
        board[y][x] = player
    print(board)

def check_line(line, player):
    # Returns True if all elements in the line match the player aka X or O
    return np.all(line == player)

def check_draw(board):
    if np.all(board != 0):
        return True
    return False
def check_win(board):
    global player
    #Check rows
    for row in range(3):
        if check_line(board[row, :], player):
            return True
    #Check Columns
    for col in range(3):
        if check_line(board[:, col], player):
            return True
    #Check diagonal
    if check_line(np.diag(board), player):
        return True
    #Check other diagonal
    if check_line(np.diag(np.fliplr(board)), player):
        return True
    #No win
    return False

def clear_board():
    global player
    global board
    board = np.full((3, 3), 0 ,dtype=int)
    player = 1

def available_moves(board):
    open_space = np.argwhere(board == 0)
    moves = []
    count = 0
    for m in open_space:
        arr = np.full((len(open_space)), 0 ,dtype=int)
        arr[count] = 1
        moves.append(arr.tolist())
        count += 1
    return moves


def draw_board(board, opponent:str):
    plt.figure(figsize=(5, 5))
    plt.xlim(0, 3)
    plt.ylim(0, 3)

    plt.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
    
    # Vertical
    plt.plot([1,1], [0,3], color='black', linewidth=2)
    plt.plot([2,2], [0,3], color='black', linewidth=2)
    # Horizontal
    plt.plot([0,3], [1,1], color='black', linewidth=2)
    plt.plot([0,3], [2,2], color='black', linewidth=2)

    #Place things
    for row in range(3):
        for col in range(3):
            symbol = board[row, col]

            if symbol != 0:
                x_pos = col + 0.5
                y_pos = 2.5 - row
                #Color
                color = 'red' if symbol == 1 else 'blue'
                #draw
                plt.text(x_pos, y_pos,'X' if symbol == 1 else 'O', fontsize=40, ha='center', va='center', color=color)
    plt.title("Tic Tac Toe vs " + opponent)
    plt.show()


# make_move(board, (0,0))
# make_move(board, (0,1))
# make_move(board, (0,2))
# available_moves(board)
# print(check_win(board))
# print(check_draw(board))
# #clear_board()
# draw_board(board, "MEEEEEEEEEEE")
# # while True:
# #     input()

[[1 0 0]
 [0 0 0]
 [0 0 0]]
[[1 0 0]
 [1 0 0]
 [0 0 0]]
[[1 0 0]
 [1 0 0]
 [1 0 0]]
[[0 1]
 [0 2]
 [1 1]
 [1 2]
 [2 1]
 [2 2]]
6
[[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]]


[(np.int64(0), np.int64(1)),
 (np.int64(0), np.int64(2)),
 (np.int64(1), np.int64(1)),
 (np.int64(1), np.int64(2)),
 (np.int64(2), np.int64(1)),
 (np.int64(2), np.int64(2))]

## Policy things

In [7]:
#Load perfect strategy
perfect_strategy = pickle.load(open("perfectPolicy.p", "rb"))
print(len(perfect_strategy))
type(perfect_strategy[(0,0,0,0,0,0,0,0,0)])

4520


numpy.ndarray

In [ ]:
class QAgent:
    def __init__(self, symbol, epsilon=0.1, gamma=0.95, alpha=0.1, policy={}):
        self.symbol = symbol
        self.q_table = policy
        self.epsilon = epsilon # Chansen att agenten gör ett slumpmässigt drag 
        self.gamma = gamma # Hur mycket framtida belöningar värderas jämfört med omedelbara.
        self.alpha = alpha # Inlärningshastighet
        self.last_state = None
        self.last_action = None

    def generate_state_string(self, board):
        return str(board.reshape(9)) # gör om numpy array till en string för användan i dict
    
    def choose_move(self, board):
        state = self.generate_state_string(board)
        moves = available_moves(board)

        #epsilon random exploration thing
        if random.random() < self.epsilon:
            return random.choice(moves)
        
        # utvärdera vilket drag man ska gör utifrån q tabellen

        # lägger alla drag som går att göra tillsamans med q värdet i en tupel och sedan lägger tupeln i en lista
        moves_with_q_value = []
        max_q = 0.0
        for m in available_moves:
            q_val = self.q_table.get((state, m), 0.0)
            moves_with_q_value.append(m, q_val)
            if q_val > max_q:
                max_q = q_val
        
        #filtrerar ut dom bästa dragen baserat på q värde
        best_moves = []
        for m in moves_with_q_value:
            if m[1] == max_q:
                best_moves.append(m)
            
            
        


## Game Player

In [ ]:
def input_processor(move):
    move = move.split(' ')
    
    x = int(move[0])
    y = int(move[1])
    return (x,y)


opponent = "Human"
choice = input("1 = Human. 2 = Random. 3 = Perfect Policy. 4 = Home Grown Policy 5 = Dont play")
run_game = True
if choice == "5":
    run_game = False
while run_game:
    
    #First Player
    player = 1
    if choice == "1":
        opponent = "Human"
        make_move(board, input_processor(input("X players turn (input like this:1 2)")))
        draw_board(board, opponent)
        if check_win(board):
            run_game = False
            print("X player win")
            clear_board()
            continue
        if check_draw(board):
            run_game = False
            print("Draw")
            clear_board()
            continue
    elif choice == "2":
        opponent = "RNGesus"
        valid_play = False
        while not valid_play:
            x = random.randint(0, 2)
            y = random.randint(0, 2)
            if board[y][x] == 0:
                valid_play = True
                make_move(board, (x,y))
                draw_board(board, opponent)
                
        if check_win(board):
            run_game = False
            print("X player win")
            clear_board()
            continue            
        if check_draw(board):                
            run_game = False
            print("Draw")
            clear_board()
            continue

    
    #Second Player
    player = 2
    make_move(board, input_processor(input("O players turn (input like this:1 2)")))
    draw_board(board, opponent)
    if check_win(board):
        run_game = False
        print("O player win")
        clear_board()
    if check_draw(board):
        run_game = False
        print("Draw")
        clear_board()
        



ValueError: invalid literal for int() with base 10: ''